In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install tifffile

In [ ]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

In [ ]:
import tifffile as tff
import matplotlib.pyplot as plt

image_path = '/kaggle/input/histopathologic-cancer-detection/train/00043555aad4d531c165e1129d281aaca9316c45.tif'

image = tff.imread(image_path)

plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
image.shape

In [ ]:
def train_img_path(id_str):
    return os.path.join(r"/kaggle/input/histopathologic-cancer-detection/train", f"{id_str}.tif")

In [ ]:
train_labels = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')

In [ ]:
train_labels.head()

In [ ]:
print(train_img_path(train_labels['id'][0]))

In [ ]:
image = tff.imread('/kaggle/input/histopathologic-cancer-detection/train/f38a6374c348f90b587e046aac6079959adf3835.tif')

plt.imshow(image)
plt.show()

In [ ]:
train_labels['filename'] = train_labels['id'].apply(train_img_path)

In [ ]:
train_labels.head()

In [ ]:
sample_data = np.empty((100, 96, 96, 3), dtype=np.uint8)
sample_labels = np.empty(100, dtype=np.int8)

for i in range(len(train_labels))[:100]:
    img_path = train_img_path(train_labels['id'][i])
    img = tff.imread(img_path)
    sample_data[i] = np.array(img)
    sample_labels[i] = train_labels['label'][i]

In [ ]:
print('No cancer images')

random_images = np.random.choice(sample_data[sample_labels == 0].shape[0], 12, replace=False)
grid = int(np.ceil(np.sqrt(12)))

fig, axs = plt.subplots(grid, grid, figsize=(5, 5))

for i, ax in enumerate(axs.flatten()):
    if i < 12:
        ax.imshow(sample_data[sample_labels == 0][random_images[i]])
        ax.axis('off') 
    else:
        fig.delaxes(ax) 

plt.tight_layout()
plt.show()

In [ ]:
print('Cancer images')

random_images = np.random.choice(sample_data[sample_labels == 1].shape[0], 12, replace=False)
grid = int(np.ceil(np.sqrt(12)))

fig, axs = plt.subplots(grid, grid, figsize=(5, 5))

for i, ax in enumerate(axs.flatten()):
    if i < 12:
        ax.imshow(sample_data[sample_labels == 1][random_images[i]])
        ax.axis('off') 
    else:
        fig.delaxes(ax) 

plt.tight_layout()
plt.show()

In [ ]:
test_path = "/kaggle/input/histopathologic-cancer-detection/test"
test_ids = [filename[:-4] for filename in os.listdir(test_path)]
test_filenames = [os.path.join(test_path, filename) for filename in os.listdir(test_path)]
test_df = pd.DataFrame()
test_df["id"] = test_ids
test_df["filename"] = test_filenames

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    rescale=1/255,
    validation_split=0.2
)

In [ ]:
train_labels['label'] = train_labels['label'].astype(str)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    shuffle = True,
    dataframe = train_labels,
    x_col = "filename",
    y_col = "label",
    target_size = (96, 96),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "binary",
    subset = "training",
    validate_filenames = False,
    seed = 10
)

validation_generator = datagen.flow_from_dataframe(
    shuffle = True,
    dataframe=train_labels,
    x_col = "filename",
    y_col = "label",
    target_size=(96, 96),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "binary",
    subset = "validation",
    validate_filenames = False,
    seed = 10
)

In [ ]:
test_generator = datagen.flow_from_dataframe(
    dataframe = test_df,
    x_col = "filename",
    y_col = None,
    target_size = (96, 96),
    color_mode = "rgb",
    batch_size = 64,
    shuffle = False,
    class_mode = None,
    validate_filenames = False,
    seed = 10
)

In [ ]:
batch_size = 32
epochs = 200
num_classes = 2

version = 1
n = 3

if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2


model_type = "ResNet % dv % d" % (depth, version)

input_shape = train_generator.image_shape

In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
def residual_block(
    inputs,
    num_filters=16,
    kernel_size=3,
    strides=1,
    activation="relu",
    batch_normalization=True,
    conv_first=True
):
    conv = Conv2D(
        num_filters,
        kernel_size=kernel_size,
        strides=strides,
        padding="same",
        kernel_initializer="he_normal",
        kernel_regularizer=l2(1e-4)
    )
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [ ]:
def ResidualNetwork(input_shape, depth, num_classes=2):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n + 2 (eg 20, 32, 44 in [a])')
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    
    inputs = Input(shape=input_shape)
    x = residual_block(inputs=inputs)
    
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:
                strides = 2
            y = residual_block(inputs=x, num_filters=num_filters, strides=strides)
            y = residual_block(inputs=y, num_filters=num_filters, activation=None)
            if stack > 0 and res_block == 0:
                x = residual_block(inputs=x, num_filters=num_filters, kernel_size=1, strides=strides, activation=None, batch_normalization=None)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(1, activation='sigmoid', kernel_initializer='he_normal')(y)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
input_shape

In [ ]:
model = ResidualNetwork(input_shape=input_shape, depth=depth)

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=Adam(learning_rate=lr_schedule(0)), metrics=['Accuracy'])
model.summary()